In [9]:
import pandas as pd

# Load the CSV
df = pd.read_csv('donnees_vol_mitm_variable.csv')  # use your actual filename if different
df['is_attack'] = 0
# List of columns to keep
columns_to_keep = [
    'mission_id', 'time', 'real_x', 'real_y', 'real_z',
    'vx', 'vy', 'vz', 'ax', 'ay', 'az',
    'roll', 'pitch', 'yaw',
    'wind_x', 'wind_y', 'wind_z',
    'erreur_m', 'velocity',
    'rain', 'fog','snow' ,'is_attack'      

]

# Filter the DataFrame
filtered_df = df[columns_to_keep]

# Save the result
filtered_df.to_csv('donnees_nettoyee_R.csv', index=False)

print(" Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'")

 Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'


In [10]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('donnees_nettoyee_R.csv')

# Final list of valid flight rows
output_rows = []

# Track current block of rows
current_block = []
current_id = None

# Counter for assigning new clean mission_ids
new_flight_id = 1

# Go through rows one by one
for _, row in df.iterrows():
    row_id = row['mission_id']

    if current_id is None:
        current_id = row_id
        current_block.append(row)
    elif row_id == current_id:
        current_block.append(row)
    else:
        # ID changed, time to process the current block
        if len(current_block) == 21:
            for r in current_block:
                r['mission_id'] = new_flight_id
                output_rows.append(r)
            new_flight_id += 1

        # Start a new block
        current_block = [row]
        current_id = row_id

# Handle the last block
if len(current_block) == 21:
    for r in current_block:
        r['mission_id'] = new_flight_id
        output_rows.append(r)

# Convert to DataFrame
output_df = pd.DataFrame(output_rows)

# Save to CSV
output_df.to_csv('donnees_vol_multiple_filtered_R.csv', index=False)

print("✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'")

✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'


In [11]:
import pandas as pd

# Charger les données
df = pd.read_csv("donnees_vol_multiple_filtered_R.csv")

# Convertir 'erreur_m' en numérique (ignore les erreurs de conversion)
df["erreur_m"] = pd.to_numeric(df["erreur_m"], errors="coerce")

# Liste des vols valides
vols_valides = []

# Vérification des conditions sur chaque vol
for mission_id, groupe in df.groupby("mission_id"):
    groupe_sorted = groupe.sort_values("time")
    premiers_wp = groupe_sorted.head(7)

    # Vérifier les deux conditions
    condition_erreur_7points = (premiers_wp["erreur_m"] > 8).any()
    moyenne_vol = groupe_sorted["erreur_m"].mean()
    condition_moyenne = moyenne_vol >= 18

    # Appliquer le OU : on supprime le vol si l'une des deux conditions est vraie
    if condition_erreur_7points or condition_moyenne:
        continue

    # Sinon, garder ce vol
    vols_valides.append(mission_id)

# Filtrer le DataFrame avec les vols valides
df_final = df[df["mission_id"].isin(vols_valides)]

# Sauvegarder dans un nouveau fichier CSV
df_final.to_csv("donnees_vols_valides_R.csv", index=False)

# Affichage de la moyenne d'erreur et du nombre de vols gardés
moyenne_finale = df_final["erreur_m"].mean()
print(f"Moyenne finale des erreurs : {moyenne_finale:.2f} mètres")
print(f"Nombre de vols restants : {len(vols_valides)}")
print("Fichier 'donnees_vols_valides_R.csv' enregistré avec succès.")


Moyenne finale des erreurs : 5.11 mètres
Nombre de vols restants : 210
Fichier 'donnees_vols_valides_R.csv' enregistré avec succès.


In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np

def main():
    # 1. Charger les données
    df = pd.read_csv('donnees_vols_valides_R.csv')

    # 2. Calcul de la matrice de corrélation
    corr = df.corr()
    print("Matrice de corrélation :")
    print(corr)
    print("\n" + "="*80 + "\n")

    # 3. Extraction des paires de variables (sans doublons ni diagonale)
    mask = np.triu(np.ones(corr.shape), k=1).astype(bool)
    corr_pairs = (
        corr.where(mask)
            .stack()
            .reset_index()
            .rename(columns={'level_0': 'Variable1', 'level_1': 'Variable2', 0: 'Corrélation'})
    )

    # 4. Filtrage des paires avec corrélation >= 0.7 en valeur absolue
    high_corr = corr_pairs[corr_pairs['Corrélation'].abs() >= 0.7] \
        .sort_values(by='Corrélation', ascending=False)

    # 5. Affichage des paires à forte corrélation
    print("Paires de variables avec |corrélation| ≥ 0.7 :")
    for _, row in high_corr.iterrows():
        v1, v2, r = row['Variable1'], row['Variable2'], row['Corrélation']
        print(f"  - {v1} ↔ {v2} : corrélation = {r:.3f}")

if __name__ == "__main__":
    main()


Matrice de corrélation :
            mission_id      time    real_x    real_y    real_z        vx  \
mission_id    1.000000 -0.466862 -0.020877  0.017718  0.023189 -0.075033   
time         -0.466862  1.000000  0.044547 -0.117438 -0.278788  0.001362   
real_x       -0.020877  0.044547  1.000000 -0.461229 -0.077033  0.565509   
real_y        0.017718 -0.117438 -0.461229  1.000000  0.206076 -0.053214   
real_z        0.023189 -0.278788 -0.077033  0.206076  1.000000 -0.015975   
vx           -0.075033  0.001362  0.565509 -0.053214 -0.015975  1.000000   
vy            0.114216 -0.192477 -0.154724  0.467739  0.185287 -0.323157   
vz            0.009958 -0.022143  0.357701 -0.084765  0.524293  0.241514   
ax           -0.067150 -0.044101  0.307508  0.190131 -0.113982  0.619942   
ay           -0.464456  0.986775  0.025523 -0.102256 -0.263967 -0.021609   
az            0.449503 -0.988545 -0.043395  0.115164  0.277414  0.007366   
roll         -0.004891  0.016247  0.007127 -0.000898 -0.024890 

In [20]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("donnees_vol_multiple_filtered_R.csv")

# Réinitialiser l'index
df = df.reset_index(drop=True)

# Garder la valeur is_attack originale par mission_id
is_attack_values = df.groupby('mission_id')['is_attack'].first().reset_index()

# Ajouter un compteur par mission_id (position du waypoint)
df['ligne_id'] = df.groupby('mission_id').cumcount()

# Passer de long à wide : chaque ligne devient un vol à plat
df_flat = df.pivot(index='mission_id', columns='ligne_id')

# Aplatir les colonnes multi-niveaux
df_flat.columns = [f"{col}_{i}" for col, i in df_flat.columns]

# Réinitialiser l'index pour fusionner
df_flat = df_flat.reset_index()

# Fusionner avec les valeurs originales de is_attack
df_flat = pd.merge(df_flat, is_attack_values, on='mission_id', how='left')

# Supprimer toutes les colonnes is_attack_X qui ont été créées par le pivot
cols_to_drop = [col for col in df_flat.columns if col.startswith('is_attack_')]
df_flat = df_flat.drop(columns=cols_to_drop)

# Sauvegarder le fichier aplati
df_flat.to_csv("vols_aplatis_R.csv", index=False)

print("✅ Vols aplatis avec succès ! (avec un seul is_attack par vol)")

✅ Vols aplatis avec succès ! (avec un seul is_attack par vol)
